In [1]:
import os, sys
from pathlib import Path

CURRENT_DIR = Path(os.getcwd())
PJROOT_DIR = CURRENT_DIR.parent.parent
LIB_PACKAGE_MAN_DIR = PJROOT_DIR / "airflow"
NOTEBOOK_LIB_DIR = PJROOT_DIR / "notebooks"
SETTINGS_DIR = PJROOT_DIR / "settings_package_man"

sys.path.append(str(LIB_PACKAGE_MAN_DIR))
sys.path.append(str(SETTINGS_DIR))

In [2]:
# import objects
from hostlist import MANAGED_HOSTS_DCT
from settings_ansible import ANSIBLE_CONF

import pandas as pd
import re
import yaml
import os
import csv
import json
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime as dt
import shutil
import os
import filecmp
import tarfile
import subprocess
from packaging import version
from IPython.display import display, HTML

# global variables
PLAYBOOK_DIR = PJROOT_DIR / "playbook"
SETTINGS_DIR = PJROOT_DIR / "settings_package_man"
DATA_DIR = PJROOT_DIR / "data_package_man"
HOSTS_OUTPUT_DIR = DATA_DIR / "hosts"
PACKAGES_OUTPUT_DIR =  DATA_DIR / "packages"
INVENTORY_FNAME = ANSIBLE_CONF["inventory_fname"]
HOSTNAME_WITH_IP = ANSIBLE_CONF["hostname_with_ip"]    
GET_OS_VERSION_PLAYBOOK_FNAME = ANSIBLE_CONF["get_os_version_playbook_fname"]
GET_PACKAGES_PLAYBOOK_FNAME = ANSIBLE_CONF["get_packages_playbook_fname"]
PACKAGE_DATA_FNAME = "package_list_data.parquet"
# 個別playbooks
UPGRADE_PACKAGE_PLAYBOOK_DIR = PLAYBOOK_DIR / "upgrade_package_playbook"
UPGRADE_PACKAGE_PLAYBOOK_FNAME = ANSIBLE_CONF["upgrade_package_playbook_fname"]

from lib_package_man import PackageManBackup, PackageList, MakePlaybook

In [3]:
self = MakePlaybook()

In [4]:
self.df.index

Index(['192.168.0.100', '192.168.0.101', '192.168.0.134', '192.168.0.137',
       '192.168.0.145', '192.168.0.135'],
      dtype='object', name='ssh_ip_address')

In [5]:
self.df

,ssh_auth_method,ssh_user,ssh_password,ssh_secretkey_path,name,package_manager,env
ssh_ip_address,,,,,,,
192.168.0.100,password,$ANSIBLE_USER,$ANSIBLE_SSH_PASS,$ANSIBLE_SSH_PRIVATE_KEY_FILE,home_gw,apt,dev
192.168.0.101,password,$ANSIBLE_USER,$ANSIBLE_SSH_PASS,NaN,stdb1,apt,dev
192.168.0.134,password,$ANSIBLE_USER,$ANSIBLE_SSH_PASS,NaN,grafana1,apt,dev
192.168.0.137,password,$ANSIBLE_USER,$ANSIBLE_SSH_PASS,NaN,influxdb2,apt,dev
192.168.0.145,password,$ANSIBLE_USER,$ANSIBLE_SSH_PASS,$ANSIBLE_SSH_PRIVATE_KEY_FILE,django05,apt,dev
192.168.0.135,password,$ANSIBLE_USER,$ANSIBLE_SSH_PASS,$ANSIBLE_SSH_PRIVATE_KEY_FILE,pacmng1,apt,dev


In [6]:
# 修正
# packageでfilterする処理を追加。

package = "python3-venv"
kwargs={}
post_upgrade_task=""
service_name=""
# def make_upgrade_package_inventory_and_playbook(package):
stdout = "Call MakePlaybook.make_get_packages_playbook method"
print(stdout)

# **kwargsでupgrade対象のhostをfilterする。
self.filter_hosts(**kwargs)

# packageがinstallされたhostを抽出する
df = self.df
ins_PL = PackageList()
package_installed_host_df = ins_PL.get_unique_hostlist_df(name=package)
package_installed_hosts = package_installed_host_df["ssh_ip_address"].values.tolist()
df.reset_index(inplace=True, drop=False)
df = df[df["ssh_ip_address"].isin(package_installed_hosts)]
df.set_index("ssh_ip_address", inplace=True, drop=True)
self.df = df
self.df

Call MakePlaybook.make_get_packages_playbook method
no filter data
Index(['ssh_ip_address', 'ssh_auth_method', 'ssh_user', 'ssh_password',
       'ssh_secretkey_path', 'name', 'package_manager', 'env'],
      dtype='object')


,ssh_auth_method,ssh_user,ssh_password,ssh_secretkey_path,name,package_manager,env
ssh_ip_address,,,,,,,
192.168.0.101,password,$ANSIBLE_USER,$ANSIBLE_SSH_PASS,NaN,stdb1,apt,dev


In [7]:

playbook = []

for ssh_ip_address in self.df.index:
    hostname = self.df.loc[ssh_ip_address, "name"]
    manager = self.df.loc[ssh_ip_address, "package_manager"]
    play = self._make_package_upgrade_play(ssh_ip_address, hostname, manager, package, post_upgrade_task, service_name)
    playbook.append(play)

# 保存dirがなければ作成
os.makedirs(UPGRADE_PACKAGE_PLAYBOOK_DIR, exist_ok=True)

# filter済のself.dfから作成されたplaybookファイルをUPGRADE_PACKAGE_PLAYBOOK_DIRに保存
output = yaml.dump(playbook, sort_keys=False)
fpath = str(UPGRADE_PACKAGE_PLAYBOOK_DIR/UPGRADE_PACKAGE_PLAYBOOK_FNAME)
with open(str(fpath), "w") as f:
    f.write(output)
stdout = f"Playbook {fpath} was made"
print(stdout)

# filter済のself.dfからinventoryファイルを作成して、UPGRADE_PACKAGE_PLAYBOOK_DIRに保存
fPath = UPGRADE_PACKAGE_PLAYBOOK_DIR / INVENTORY_FNAME
self.make_inventory(fPath)


Playbook /home/ogs-digilife/package_man/playbook/upgrade_package_playbook/upgrade-package.yml was made
Call MakePlaybook.make_inventory method
Inventory file /home/ogs-digilife/package_man/playbook/upgrade_package_playbook/inventory.yml was made

